Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from catboost import CatBoostClassifier, cv, Pool

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold

pd.options.display.max_columns=100

Загрузка обучающего и тестового датасетов

In [31]:
def do_cv(x_train, x_test, y_train, features, cat_features, params, splitter):
    oof_result = pd.Series([0] * x_train.shape[0], index=x_train.index)
    result = [0] * x_test.shape[0]
    scores = []
    for ind_trn, ind_test in splitter.split(x_train, y_train):
        X_train_df = x_train[features].iloc[ind_trn]
        X_valid_df = x_train[features].iloc[ind_test]
        y_train_df = y_train.iloc[ind_trn]
        y_valid_df = y_train.iloc[ind_test]
        
        print(X_train_df.columns)
        eval_dataset = Pool(X_valid_df, y_valid_df, cat_features)
        model = CatBoostClassifier(**params)

        model.fit(
            X_train_df,  
            y_train_df,
            cat_features,
            eval_set=eval_dataset,
            verbose=0,
            use_best_model=True,
        )
        
        scores.append(roc_auc_score(y_valid_df, model.predict_proba(X_valid_df)[:, 1]))
        result += model.predict_proba(x_test[features])[:, 1:].flatten()/5
        oof_result.loc[X_valid_df.index] = model.predict_proba(X_valid_df)[:, 1]
        
    return scores, result, oof_result

In [2]:
train = pd.read_csv('data/catboost_train.csv')
test = pd.read_csv('data/catboost_test.csv')

In [92]:
train['emp_length'] = train['emp_length'].astype('int')
test['emp_length'] = test['emp_length'].astype('int')

In [18]:
train

,installment,dti,funded_amnt,annual_inc,emp_length,term,inq_last_6mths,mths_since_recent_inq,delinq_2yrs,chargeoff_within_12_mths,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,id,purpose,subgrade,home_ownership,psk,percent,percent_per_year,residual_dti,loan_income_ratio,additional_dti,dti_ratio_min,dti_ratio_max,waste_to_installment_ratio,is_first_loan,is_not_sber_client,acc_open_past_24mths_ratio,tot_hi_cred_lim_funded_amnt_ratio,tot_hi_cred_lim_annual_inc_ratio,funded_amnt_tot_hi_cred_lim_ratio,annual_inc_tot_hi_cred_lim_ratio,avg_cur_bal_tot_hi_cred_lim_ratio,even_salary,even_installment,is_emp_length_unknown,is_dti_b_35,is_common_amount,funded_amnt_mod_1000,funded_amnt_mod_500,funded_amnt_mod_100,funded_amnt_mod_10,installment_log,installment_sqrt,dti_log,dti_sqrt,funded_amnt_log,funded_amnt_sqrt,annual_inc_log,annual_inc_sqrt,avg_cur_bal_log,avg_cur_bal_sqrt,tot_hi_cred_lim_log,tot_hi_cred_lim_sqrt
0,365.59,12.92,10500,66400.0,6.0,36,0,7.0,0,0,0.0,0.0,4.0,2879.0,35225.0,0,0,0,purpose_debt_consolidation,sub_grade_C2,home_ownership_RENT,13161.24,0.253451,8.4,6.312952,0.158133,19.527048,2.046586,0.661646,195.548748,False,False,0.222222,3.354762,0.530497,0.298084,1.885025,0.081732,0,0,0,0,0,500,0,0,0,5.901513,19.120408,2.558777,3.594440,9.259131,102.469508,11.103452,257.681975,7.965198,53.656314,10.469511,187.683244
1,535.54,19.11,15000,80000.0,10.0,36,1,2.0,1,0,3.0,0.0,4.0,13266.0,164512.0,0,0,1,purpose_credit_card,sub_grade_C5,home_ownership_MORTGAGE,19279.44,0.285296,9.5,11.076900,0.187500,27.143100,1.725212,0.704046,237.890727,False,False,0.173913,10.967467,2.056400,0.091179,0.486287,0.080638,1,0,0,0,1,0,0,0,0,6.283276,23.141737,2.950212,4.371499,9.615805,122.474487,11.289782,282.842712,9.492960,115.178123,12.010739,405.600789
2,517.16,26.55,22400,50000.0,10.0,60,4,2.0,0,0,0.0,0.0,3.0,12445.0,163277.0,0,0,2,purpose_credit_card,sub_grade_C1,home_ownership_MORTGAGE,31029.60,0.385250,7.7,14.138160,0.448000,38.961840,1.877896,0.681436,213.908655,False,False,0.130435,7.289152,3.265540,0.137190,0.306228,0.076220,1,0,0,0,0,400,400,0,0,6.248352,22.741152,3.279030,5.152669,10.016816,149.666295,10.819778,223.606798,9.429074,111.557160,12.003203,404.075488
3,342.66,5.74,9800,41000.0,0.0,36,1,4.0,1,0,0.0,0.0,5.0,1936.0,37600.0,0,0,3,purpose_debt_consolidation,sub_grade_C5,home_ownership_RENT,12335.76,0.258751,8.6,-4.289073,0.239024,15.769073,-1.338284,0.364004,57.233604,False,False,0.238095,3.836735,0.917073,0.260638,1.090426,0.051489,1,0,0,0,0,800,300,0,0,5.836739,18.511078,1.747461,2.395830,9.190138,98.994949,10.621327,202.484567,7.568379,44.000000,10.534759,193.907194
4,381.46,4.88,17325,74000.0,10.0,60,1,6.0,0,0,0.0,0.0,3.0,35056.0,190262.0,0,0,4,purpose_home_improvement,sub_grade_B3,home_ownership_MORTGAGE,22887.60,0.321074,6.4,-1.305838,0.234122,11.065838,-3.737064,0.440997,78.889879,False,False,0.157895,10.981934,2.571108,0.091059,0.388937,0.184251,1,0,0,0,0,325,325,25,5,5.944006,19.531001,1.585147,2.209072,9.759906,131.624466,11.211820,272.029410,10.464702,187.232476,12.156157,436.190325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1002.03,13.29,30000,150000.0,10.0,36,0,2.0,1,0,0.0,0.0,3.0,3992.0,110852.0,0,0,29995,purpose_other,sub_grade_C1,home_ownership_OWN,36073.08,0.202436,6.7,5.273760,0.200000,21.306240,2.520024,0.623761,165.788449,False,False,0.130435,3.695067,0.739013,0.270631,1.353156,0.036012,1,0,0,0,1,0,0,0,0,6.909783,31.654857,2.587013,3.645545,10.308953,173.205081,11.918391,387.298335,8.292048,63.182276,11.615951,332.944440
29996,373.92,29.12,15000,60000.0,9.0,60,0,11.0,0,0,0.0,0.0,2.0,7482.0,70344.0,0,0,29996,purpose_debt_consolidation,sub_grade_D4,home_ownership_RENT,22435.20,0.495680,9.9,21.641600,0.250000,36.598400,1.345557,0.795663,389.388104,False,Fal

In [93]:
targ_cols = [i for i in train.columns if i not in ['def', 'id'] ]

X_train = train[targ_cols]
X_test = test[targ_cols]

y_train = train["def"]

In [4]:
targ_cols

['installment',
 'dti',
 'funded_amnt',
 'annual_inc',
 'emp_length',
 'term',
 'inq_last_6mths',
 'mths_since_recent_inq',
 'delinq_2yrs',
 'chargeoff_within_12_mths',
 'num_accts_ever_120_pd',
 'num_tl_90g_dpd_24m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'delinq_amnt',
 'purpose',
 'subgrade',
 'home_ownership',
 'psk',
 'percent',
 'percent_per_year',
 'residual_dti',
 'loan_income_ratio',
 'additional_dti',
 'dti_ratio_min',
 'dti_ratio_max',
 'waste_to_installment_ratio',
 'is_first_loan',
 'is_not_sber_client',
 'acc_open_past_24mths_ratio',
 'tot_hi_cred_lim_funded_amnt_ratio',
 'tot_hi_cred_lim_annual_inc_ratio',
 'funded_amnt_tot_hi_cred_lim_ratio',
 'annual_inc_tot_hi_cred_lim_ratio',
 'avg_cur_bal_tot_hi_cred_lim_ratio',
 'even_salary',
 'even_installment',
 'is_emp_length_unknown',
 'is_dti_b_35',
 'is_common_amount',
 'funded_amnt_mod_1000',
 'funded_amnt_mod_500',
 'funded_amnt_mod_100',
 'funded_amnt_mod_10',
 'installment_log',
 'installment_sqrt'

In [23]:
train['installment'].dtype == 'O'

False

In [71]:
params = {
    "iterations": 1000,
    "depth": 1,
    "learning_rate": 0.08,
#     "l2_leaf_reg": 20, # 50 
    ""
    "loss_function": "Logloss",
    "verbose": False,
    "eval_metric": 'AUC',
    "od_type": 'Iter',
    "od_wait": 150,
#     "random_strength": 1,
}

init_cols = [
#     'percent',
#     'annual_inc',
#     'acc_open_past_24mths',
#     'installment',
#     'avg_cur_bal',
#     'mths_since_recent_inq',
#     'tot_hi_cred_lim',
#     'dti',
    
    'term',
    'acc_open_past_24mths', # 677 02
    'funded_amnt',
    'percent_per_year',
    'additional_dti', #671
    'num_accts_ever_120_pd',
    'annual_inc', # 682 024,
    'tot_hi_cred_lim',
    'purpose', # 6839 035
    'home_ownership', # 683 027
    'avg_cur_bal', # 684 041
]

cat_features = [
    'purpose',
    'home_ownership',
]

rs = {}

skf = StratifiedKFold(n_splits=5, random_state=18, shuffle=True)
for feat in targ_cols:
    print(feat)
    if feat in init_cols:
        continue
        
    if train[feat].dtype == 'O':
        scores, result, oof_result = do_cv(X_train, X_test, y_train, init_cols + [feat], cat_features + [feat], params, skf)
    else:
        scores, result, oof_result = do_cv(X_train, X_test, y_train, init_cols + [feat], cat_features, params, skf)
    print(feat, np.mean(scores), np.std(scores))
    rs[feat] = [np.mean(scores), np.std(scores)]

installment
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'installment'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'installment'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'installment'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'installment']

Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'chargeoff_within_12_mths'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'chargeoff_within_12_mths'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'chargeoff_within_12_mths'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_

Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'residual_dti'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'residual_dti'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'residual_dti'],
      dtype='object')
residual_dti 0.6846186139844405 0.008241859667568035
loan_income_ratio
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'pu

Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'is_not_sber_client'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'is_not_sber_client'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'is_not_sber_client'],
      dtype='object')
is_not_sber_client 0.685268894092378 0.00863788708604777
acc_open_past_24mths_ratio
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc'

Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'avg_cur_bal_tot_hi_cred_lim_ratio'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'avg_cur_bal_tot_hi_cred_lim_ratio'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'avg_cur_bal_tot_hi_cred_lim_ratio'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 

Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'funded_amnt_mod_1000'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'funded_amnt_mod_1000'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'funded_amnt_mod_1000'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
      

Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'dti_log'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'dti_log'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'dti_log'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'dti_log'],
      dtype='object')
dti_

Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'avg_cur_bal_log'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'avg_cur_bal_log'],
      dtype='object')
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cred_lim', 'purpose', 'home_ownership', 'avg_cur_bal',
       'avg_cur_bal_log'],
      dtype='object')
avg_cur_bal_log 0.6850569252805296 0.008480926621125439
avg_cur_bal_sqrt
Index(['term', 'acc_open_past_24mths', 'funded_amnt', 'percent_per_year',
       'additional_dti', 'num_accts_ever_120_pd', 'annual_inc',
       'tot_hi_cre

In [72]:
pd.DataFrame(rs).T.sort_values(0).tail(10)

,0,1
is_first_loan,0.685109,0.008535
funded_amnt_mod_10,0.685112,0.008725
is_common_amount,0.685123,0.008355
even_installment,0.685239,0.008655
is_dti_b_35,0.685255,0.008637
even_salary,0.685267,0.008707
is_not_sber_client,0.685269,0.008638
mths_since_recent_inq,0.685357,0.008281
is_emp_length_unknown,0.685366,0.008828
funded_amnt_mod_100,0.685441,0.008068


In [47]:
pd.DataFrame(rs).T.sort_values(0).tail(10)

,0,1
annual_inc_tot_hi_cred_lim_ratio,0.684866,0.008781
tot_hi_cred_lim_funded_amnt_ratio,0.684924,0.008605
is_emp_length_unknown,0.684935,0.009156
is_common_amount,0.684957,0.008453
loan_income_ratio,0.684959,0.008423
psk,0.685005,0.009214
tot_hi_cred_lim,0.685057,0.009019
tot_hi_cred_lim_annual_inc_ratio,0.685097,0.008713
funded_amnt_mod_100,0.685123,0.008677
funded_amnt_mod_10,0.685124,0.008483


In [82]:
train

,installment,dti,funded_amnt,annual_inc,emp_length,term,inq_last_6mths,mths_since_recent_inq,delinq_2yrs,chargeoff_within_12_mths,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,id,purpose,subgrade,home_ownership,psk,percent,percent_per_year,residual_dti,loan_income_ratio,additional_dti,dti_ratio_min,dti_ratio_max,waste_to_installment_ratio,is_first_loan,is_not_sber_client,acc_open_past_24mths_ratio,tot_hi_cred_lim_funded_amnt_ratio,tot_hi_cred_lim_annual_inc_ratio,funded_amnt_tot_hi_cred_lim_ratio,annual_inc_tot_hi_cred_lim_ratio,avg_cur_bal_tot_hi_cred_lim_ratio,even_salary,even_installment,is_emp_length_unknown,is_dti_b_35,is_common_amount,funded_amnt_mod_1000,funded_amnt_mod_500,funded_amnt_mod_100,funded_amnt_mod_10,installment_log,installment_sqrt,dti_log,dti_sqrt,funded_amnt_log,funded_amnt_sqrt,annual_inc_log,annual_inc_sqrt,avg_cur_bal_log,avg_cur_bal_sqrt,tot_hi_cred_lim_log,tot_hi_cred_lim_sqrt
0,365.59,12.92,10500,66400.0,6.0,36,0,7.0,0,0,0.0,0.0,4.0,2879.0,35225.0,0,0,0,purpose_debt_consolidation,sub_grade_C2,home_ownership_RENT,13161.24,0.253451,8.4,6.312952,0.158133,19.527048,2.046586,0.661646,195.548748,False,False,0.222222,3.354762,0.530497,0.298084,1.885025,0.081732,0,0,0,0,0,500,0,0,0,5.901513,19.120408,2.558777,3.594440,9.259131,102.469508,11.103452,257.681975,7.965198,53.656314,10.469511,187.683244
1,535.54,19.11,15000,80000.0,10.0,36,1,2.0,1,0,3.0,0.0,4.0,13266.0,164512.0,0,0,1,purpose_credit_card,sub_grade_C5,home_ownership_MORTGAGE,19279.44,0.285296,9.5,11.076900,0.187500,27.143100,1.725212,0.704046,237.890727,False,False,0.173913,10.967467,2.056400,0.091179,0.486287,0.080638,1,0,0,0,1,0,0,0,0,6.283276,23.141737,2.950212,4.371499,9.615805,122.474487,11.289782,282.842712,9.492960,115.178123,12.010739,405.600789
2,517.16,26.55,22400,50000.0,10.0,60,4,2.0,0,0,0.0,0.0,3.0,12445.0,163277.0,0,0,2,purpose_credit_card,sub_grade_C1,home_ownership_MORTGAGE,31029.60,0.385250,7.7,14.138160,0.448000,38.961840,1.877896,0.681436,213.908655,False,False,0.130435,7.289152,3.265540,0.137190,0.306228,0.076220,1,0,0,0,0,400,400,0,0,6.248352,22.741152,3.279030,5.152669,10.016816,149.666295,10.819778,223.606798,9.429074,111.557160,12.003203,404.075488
3,342.66,5.74,9800,41000.0,0.0,36,1,4.0,1,0,0.0,0.0,5.0,1936.0,37600.0,0,0,3,purpose_debt_consolidation,sub_grade_C5,home_ownership_RENT,12335.76,0.258751,8.6,-4.289073,0.239024,15.769073,-1.338284,0.364004,57.233604,False,False,0.238095,3.836735,0.917073,0.260638,1.090426,0.051489,1,0,0,0,0,800,300,0,0,5.836739,18.511078,1.747461,2.395830,9.190138,98.994949,10.621327,202.484567,7.568379,44.000000,10.534759,193.907194
4,381.46,4.88,17325,74000.0,10.0,60,1,6.0,0,0,0.0,0.0,3.0,35056.0,190262.0,0,0,4,purpose_home_improvement,sub_grade_B3,home_ownership_MORTGAGE,22887.60,0.321074,6.4,-1.305838,0.234122,11.065838,-3.737064,0.440997,78.889879,False,False,0.157895,10.981934,2.571108,0.091059,0.388937,0.184251,1,0,0,0,0,325,325,25,5,5.944006,19.531001,1.585147,2.209072,9.759906,131.624466,11.211820,272.029410,10.464702,187.232476,12.156157,436.190325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1002.03,13.29,30000,150000.0,10.0,36,0,2.0,1,0,0.0,0.0,3.0,3992.0,110852.0,0,0,29995,purpose_other,sub_grade_C1,home_ownership_OWN,36073.08,0.202436,6.7,5.273760,0.200000,21.306240,2.520024,0.623761,165.788449,False,False,0.130435,3.695067,0.739013,0.270631,1.353156,0.036012,1,0,0,0,1,0,0,0,0,6.909783,31.654857,2.587013,3.645545,10.308953,173.205081,11.918391,387.298335,8.292048,63.182276,11.615951,332.944440
29996,373.92,29.12,15000,60000.0,9.0,60,0,11.0,0,0,0.0,0.0,2.0,7482.0,70344.0,0,0,29996,purpose_debt_consolidation,sub_grade_D4,home_ownership_RENT,22435.20,0.495680,9.9,21.641600,0.250000,36.598400,1.345557,0.795663,389.388104,False,Fal

In [102]:
train['delinq_amnt'].value_counts(1)

0        0.997367
30       0.000100
25       0.000100
65000    0.000100
75       0.000067
           ...   
249      0.000033
153      0.000033
73       0.000033
1000     0.000033
4618     0.000033
Name: delinq_amnt, Length: 72, dtype: float64

In [ ]:
params = {
    "iterations": 2000,
    "depth": 1,
    "learning_rate": 0.07,
#     "l2_leaf_reg": 20, # 50 
    ""
    "loss_function": "Logloss",
    "verbose": False,
    "eval_metric": 'AUC',
    "od_type": 'Iter',
    "od_wait": 300,
#     "random_strength": 1,
}

columns = [
    'term',
    'acc_open_past_24mths', # 677 02
    'funded_amnt',
    'percent_per_year',
    'additional_dti', #671
    'num_accts_ever_120_pd',
    'annual_inc', # 682 024,
    'tot_hi_cred_lim',
    'purpose', # 6839 035
    'home_ownership', # 683 027
    
    'avg_cur_bal', # 684 041
    'mths_since_recent_inq',
    'installment',
#     'subgrade',
    
#     'tot_hi_cred_lim_annual_inc_ratio', # 6845 035
#     'is_first_loan', # 6846 039
#     'even_salary', # 682 019


]

cat_features = [
    'purpose',
    'home_ownership',
#     'emp_length',
]

skf = StratifiedKFold(n_splits=5, random_state=18)
oof_result = pd.Series([0] * train.shape[0])
result = [0] * test.shape[0]
scores = []
for ind_trn, ind_test in skf.split(X_train, y_train):
    X_train_df = X_train[columns].iloc[ind_trn]
    X_valid_df = X_train[columns].iloc[ind_test]
    y_train_df = y_train.iloc[ind_trn]
    y_valid_df = y_train.iloc[ind_test]

    eval_dataset = Pool(X_valid_df, y_valid_df, cat_features)

    model = CatBoostClassifier(
        **params,
#         per_float_feature_quantization=['7:border_count=1024']
    )

    model.fit(
        X_train_df,  
        y_train_df,
        cat_features,
        eval_set=eval_dataset,
        verbose=50,
        use_best_model=True,
    )
    result += model.predict_proba(test[columns])[:, 1:].flatten()/5
    scores.append(model.best_score_['validation']['AUC'])
    oof_result.loc[ind_test] = model.predict_proba(X_valid_df)[:, 1]


d:\kaggle\sber_plus_nsu\.venv\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0:	test: 0.5397239	best: 0.5397239 (0)	total: 9.45ms	remaining: 18.9s
50:	test: 0.6894413	best: 0.6894413 (50)	total: 646ms	remaining: 24.7s
100:	test: 0.6914900	best: 0.6918345 (96)	total: 1.31s	remaining: 24.6s
150:	test: 0.6925971	best: 0.6926787 (149)	total: 2.13s	remaining: 26.1s
200:	test: 0.6936722	best: 0.6936722 (200)	total: 2.91s	remaining: 26.1s
250:	test: 0.6945947	best: 0.6946128 (248)	total: 3.55s	remaining: 24.8s
300:	test: 0.6953281	best: 0.6953738 (296)	total: 4.64s	remaining: 26.2s
350:	test: 0.6957434	best: 0.6959781 (337)	total: 5.38s	remaining: 25.3s
400:	test: 0.6961284	best: 0.6961766 (397)	total: 5.99s	remaining: 23.9s
450:	test: 0.6962940	best: 0.6962940 (450)	total: 6.59s	remaining: 22.6s
500:	test: 0.6962654	best: 0.6963876 (486)	total: 7.23s	remaining: 21.6s
550:	test: 0.6961732	best: 0.6963907 (507)	total: 8.23s	remaining: 21.6s
600:	test: 0.6960369	best: 0.6963907 (507)	total: 8.84s	remaining: 20.6s
650:	test: 0.6960224	best: 0.6963907 (507)	total: 9.59s	r

In [ ]:
#depth 1, 0.07, 20
np.mean(scores), np.std(scores), roc_auc_score(y_train, oof_result)

In [86]:
#depth 1, 0.07, 20
np.mean(scores), np.std(scores), roc_auc_score(y_train, oof_result)

(0.6872142150667075, 0.006532246032829312, 0.686786879574213)

In [60]:
#depth 1, 0.07, 20
np.mean(scores), np.std(scores), roc_auc_score(y_train, oof_result)

(0.685544894152709, 0.005612101193541767, 0.6850534233585779)

Применение модели к тестовой выборке

In [243]:
test['prob'] = result

Выгрузка решения для коммита в соревновании

In [244]:
test[['id', 'prob']].to_csv('cb_submission.csv', index=False)

In [12]:
test[['id', 'prob']]

,id,prob
0,0,0.125105
1,1,0.133632
2,2,0.224207
3,3,0.189226
4,4,0.169653
...,...,...
29995,29995,0.124133
29996,29996,0.081821
29997,29997,0.263643
29998,29998,0.037455


In [238]:
test[['id', 'prob']]

,id,prob
0,0,0.142241
1,1,0.164546
2,2,0.235152
3,3,0.199765
4,4,0.170037
...,...,...
29995,29995,0.135544
29996,29996,0.079723
29997,29997,0.263769
29998,29998,0.051038
